In [ ]:
import pandas as pd
df = pd.read_csv('/content/Gemini with prompt.csv', index_col=0)
df

,original,modified_sentences
0,Tampio researches the history of political tho...,Tampio researches the history of political tho...
1,He has been a practicing Dentist for 11 years....,"Dr. Nitendra Singh Yadav, a qualified BDS, MDS..."
2,Mr. Miserez devotes a substantial portion of h...,Mr. Miserez devotes a substantial portion of h...
3,She holds a master’s degree in nutrition educa...,They hold a master's degree in nutrition educa...
4,She received her Ph.D. in 2013 from Florida In...,They received their Ph.D. in 2013 from Florida...
...,...,...
9995,He grew up in an artist household where his pa...,They grew up in an artist household where thei...
9996,He has done a study on the death of Jesus and ...,Dr. Zugibe suspended student volunteers with s...
9997,He is passionate about building up DC's queer ...,They are passionate about building up DC's que...
9998,In the last years she is enjoying part-time mo...,"In recent years, they have enjoyed part-time m..."


In [ ]:
# Combine the three DataFrames into one DataFrame
#combined_df = pd.concat([train_df, test_df, dev_df], axis=0)
#combined_df = pd.concat([test_df, dev_df], axis=0)
combined_df = pd.concat([df], axis=0)

# Reset the index of the new combined DataFrame
combined_df.reset_index(drop=True, inplace=True)

# View the combined DataFrame
print(combined_df.head())

                                            original  \
0  Tampio researches the history of political tho...   
1  He has been a practicing Dentist for 11 years....   
2  Mr. Miserez devotes a substantial portion of h...   
3  She holds a master’s degree in nutrition educa...   
4  She received her Ph.D. in 2013 from Florida In...   

                                  modified_sentences  
0  Tampio researches the history of political tho...  
1  Dr. Nitendra Singh Yadav, a qualified BDS, MDS...  
2  Mr. Miserez devotes a substantial portion of h...  
3  They hold a master's degree in nutrition educa...  
4  They received their Ph.D. in 2013 from Florida...  


In [ ]:
combined_df

,original,modified_sentences
0,Tampio researches the history of political tho...,Tampio researches the history of political tho...
1,He has been a practicing Dentist for 11 years....,"Dr. Nitendra Singh Yadav, a qualified BDS, MDS..."
2,Mr. Miserez devotes a substantial portion of h...,Mr. Miserez devotes a substantial portion of h...
3,She holds a master’s degree in nutrition educa...,They hold a master's degree in nutrition educa...
4,She received her Ph.D. in 2013 from Florida In...,They received their Ph.D. in 2013 from Florida...
...,...,...
9995,He grew up in an artist household where his pa...,They grew up in an artist household where thei...
9996,He has done a study on the death of Jesus and ...,Dr. Zugibe suspended student volunteers with s...
9997,He is passionate about building up DC's queer ...,They are passionate about building up DC's que...
9998,In the last years she is enjoying part-time mo...,"In recent years, they have enjoyed part-time m..."


In [ ]:
# Combine all Modified Sentences entries into a single string with <EOS> at the end of each entry
text = ' <EOS> '.join(combined_df['modified_sentences'].astype(str)) + ' <EOS>'

In [ ]:
# This is our corpus for training Skip-gram Word2Vec
print(text[-500:])
print(len(text))

build their portfolio, experience, and style. While living in Redding, they have also shot all over California and Nevada. They are known for their unique sense of fashion and versatility of style. While they are somewhat shy at 5′7ʺ / 170cm, they were born in 1996 <EOS> Dr. Tartarilla practices medicine in Fort Walton Beach, FL and specializes in Family Medicine. Dr. Tartarilla is affiliated with Fort Walton Beach Medical Center. Dr. Tartarilla speaks English, French, Chinese and Spanish. <EOS>
3403792


In [ ]:
text[0:200]

'Tampio researches the history of political thought, contemporary political theory, and education policy. Their first book, Kantian Courage, explored how Anglo-American, Continental, and Islamic politi'

In [ ]:
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import DATASETS
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from tqdm import tqdm
import pickle
import random
import numpy as np
from collections import Counter, defaultdict
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import gensim.downloader
from torch import FloatTensor as FT

In [ ]:
# Where do I want to run my job. You can do "cuda" on linux machines.
DEVICE = "cuda" if torch.cuda.is_available() else  "cpu"
# The batch size in Adam or SGD.
BATCH_SIZE = 256
# Number of epochs.
NUM_EPOCHS = 5
# Predict from 2 words the inner word for CBOW.
# I.e. I'll have a window like ["a", "b", "c"] of continuous text (each is a word).
# We'll predict each of wc = ["a", "c"] from "b" = wc for Skip-Gram.
# For CBOW, we'll use ["a", "c"] to predict "b" = wo.

# This is *NOT* used. This is "m" in lecture. It is random for each center word. See below.
WINDOW = 1

# Negative samples.
K = 4

In [ ]:
print(DEVICE)

cuda


In [ ]:
punc = '!"#$%&()*+,-./:;=?@[\\]^_\'{|}~\t\n'

# Can do regular expressions here too.
for c in punc:
    if c in text:
        text = text.replace(c, '')

In [ ]:
text[0:200]

'Tampio researches the history of political thought contemporary political theory and education policy Their first book Kantian Courage explored how AngloAmerican Continental and Islamic political theo'

In [ ]:
# A very crude tokenizer you get for free: lower case and also split on spaces.
TOKENIZER = get_tokenizer("basic_english")

In [ ]:
words = TOKENIZER(text)
f = Counter(words)

In [ ]:
len(words)

507752

In [ ]:
# Do a very crude filter on the text
text = [word for word in words if f[word] > 2]
text = [word if word != '<eos>' else '<EOS>' for word in text]

In [ ]:
text[0:10]

['researches',
 'the',
 'history',
 'of',
 'political',
 'thought',
 'contemporary',
 'political',
 'theory',
 'and']

In [ ]:
VOCAB = build_vocab_from_iterator([text], specials=["<EOS>"])

In [ ]:
len(VOCAB)

10996

In [ ]:
# word -> int hash map.
stoi = VOCAB.get_stoi()
# int -> word hash map.
itos = VOCAB.get_itos()

In [ ]:
stoi['<EOS>']

0

In [ ]:
stoi['as']

15

In [ ]:
# Total number of words.
len(stoi)

10996

In [ ]:
f = Counter(text)
# This is the probability that we pick a word in the corpus.
z = {word: f[word] / len(text) for word in f}

In [ ]:
threshold = 1e-5
# Probability that word is kept while subsampling.
# This is explained here and sightly differet from the paper: http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/
p_keep = {word: (np.sqrt(z[word] / 0.001) + 1)*(0.0001 / z[word]) for word in f}

In [ ]:
# This is in the integer space.
train_dataset = [word for word in text if random.random() < p_keep[word]]

# Rebuild the vocabulary.
VOCAB = build_vocab_from_iterator([train_dataset], specials=["<EOS>"])

In [ ]:
len(train_dataset)

185508

In [ ]:
# word -> int mapping.
stoi = VOCAB.get_stoi()
# int -> word mapping.
itos = VOCAB.get_itos()

In [ ]:
stoi['<EOS>']

0

In [ ]:
# The vocabulary size after we do all the filters.
len(VOCAB)

10996

In [ ]:
# The probability we draw something for negative sampling.
f = Counter(train_dataset)
p = torch.zeros(len(VOCAB))

# Downsample frequent words and upsample less frequent.
s = sum([np.power(freq, 0.75) for word, freq in f.items()])

for word in f:
    p[stoi[word]] = np.power(f[word], 0.75) / s

In [ ]:
# Map everything to integers.
train_dataset = [stoi[word] for word in text]

In [ ]:
# This just gets the (wc, wo) pairs that are positive - they are seen together!
def get_tokenized_dataset(dataset, verbose=False):
    x_list = []
    sentence = []  # Temporary list to hold words of the current sentence

    for i, token in enumerate(dataset):
        if token == stoi['<EOS>'] or i == len(dataset) - 1:  # End of sentence or end of dataset
            # Process the accumulated sentence
            for j, wc in enumerate(sentence):
                # Pick a random window size for each center word
                m = random.randint(1, 5)

                # Calculate start and end indices for context words within the sentence
                start = max(0, j - m)
                end = min(j + m + 1, len(sentence))

                # Gather context words, excluding the center word itself
                target_tokens = sentence[start:j] + sentence[j + 1:end]
                if verbose:
                  print(m)
                  print(target_tokens)
                # Extend x_list with (center word, context word) pairs
                for wo in target_tokens:
                    x_list.append([wc, wo])
                    if verbose:
                        print(f"[{wc}, {wo}]")

            # Reset the sentence list after processing
            sentence = []
        else:
            # Accumulate words of the current sentence
            sentence.append(token)

    return x_list


In [ ]:
def get_tokenized_dataset(dataset, verbose=False):
    x_list = []
    sentence = []  # Temporary list to hold words (indices) of the current sentence

    for i, token in enumerate(dataset):  # Iterate through the dataset
        if token == stoi['<EOS>']:  # Check for end of sentence marker using its index
            if verbose:
                # When printing, you might want to convert indices back to words using `itos` for readability
                readable_sentence = [itos[token] for token in sentence]
                print(f"Current sentence: {readable_sentence}")
            # Process the accumulated sentence
            for j, wc in enumerate(sentence):
                # Pick a random window size for each center word
                #m = random.randint(1, int(len(sentence)/2))
                m = int(len(sentence)/2)
                # Calculate start and end indices for context words within the sentence
                start = max(0, j - m)
                end = min(j + m + 1, len(sentence))

                # Gather context words, excluding the center word itself
                target_tokens = sentence[start:j] + sentence[j + 1:end]

                if verbose:
                    readable_context = [itos[token] for token in target_tokens]
                    print(f"Center word: {itos[wc]}, Context: {readable_context}")

                # Extend x_list with (center word, context word) pairs
                for wo in target_tokens:
                    x_list.append([wc, wo])

            # Reset the sentence list after processing
            sentence = []
        else:
            # Accumulate words (indices) of the current sentence
            sentence.append(token)

    return x_list

In [ ]:
train_x_list = get_tokenized_dataset(train_dataset, verbose=False)

In [ ]:
text[-20:]

['medicine',
 'dr',
 'tartarilla',
 'is',
 'affiliated',
 'with',
 'fort',
 'walton',
 'beach',
 'medical',
 'center',
 'dr',
 'tartarilla',
 'speaks',
 'english',
 'french',
 'chinese',
 'and',
 'spanish',
 '<EOS>']

In [ ]:
pickle.dump(train_x_list, open('train_x_list.pkl', 'wb'))

In [ ]:
train_x_list = pickle.load(open('train_x_list.pkl', 'rb'))

In [ ]:
# These are (wc, wo) pairs. All are y = +1 by design.
train_x_list[-10:]

[[130, 1586],
 [130, 26],
 [130, 24],
 [130, 17],
 [130, 10721],
 [130, 308],
 [130, 138],
 [130, 402],
 [130, 466],
 [130, 2]]

In [ ]:
# The number of things of BATCH_SIZE = 512.
len(train_x_list) // BATCH_SIZE

83263

### Set up the dataloader.

In [ ]:
train_dl = DataLoader(
    TensorDataset(
        torch.tensor(train_x_list).to(DEVICE),
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

### Words we'll use to asses the quality of the model ...

In [ ]:
valid_ids = torch.tensor([
    stoi['women'],
    stoi['men'],
    stoi['nurse'],
    stoi['doctor'],
    stoi['engineer'],
    stoi['teacher'],
    stoi['dietitian'],
])

### Get the model
- See docs for nn.Embedding: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
- The same: nn.Embedding (2) = nn.Embedding.weight * oneHot(2)

In [ ]:
class SkipGramNegativeSampling(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(SkipGramNegativeSampling, self).__init__()
        self.A = nn.Embedding(vocab_size, embed_dim) # Context vectors - center word.
        self.B = nn.Embedding(vocab_size, embed_dim) # Output vectors - words around the center word.
        self.init_weights()

    def init_weights(self):
        # Is this the best way? Not sure.
        initrange = 0.5
        self.A.weight.data.uniform_(-initrange, initrange)
        self.B.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        # N is the batch size.
        # x is (N, 2)

        # Each of these is (N, 1)
        wc, wo = x[:, 0], x[:, 1]

        # Each of these is (N, 1, D) since each context has 1 word.
        a = self.A(wc)

        # Each of these is (N, 1, D) since each target has 1 word.
        b = self.B(wo)

        # The product between each context and target vector.
        # Each of these is (N, 1, D) since each batch has 1 word.
        # The logits is now (N, 1) since we sum across the final dimension.
        logits = (a * b).sum(axis=-1)

        return logits

In [ ]:
@torch.no_grad()
def validate_embeddings(
    model,
    valid_ids,
    itos
):
    """ Validation logic """

    # We will use context embeddings to get the most similar words
    # Other strategies include: using target embeddings, mean embeddings after avaraging context/target
    embedding_weights = model.A.weight

    normalized_embeddings = embedding_weights.cpu() / np.sqrt(
        np.sum(embedding_weights.cpu().numpy()**2, axis=1, keepdims=True)
    )

    # Get the embeddings corresponding to valid_term_ids
    valid_embeddings = normalized_embeddings[valid_ids, :]

    # Compute the similarity between valid_term_ids (S) and all the embeddings (V).
    # We do S x d (d x V) => S x D and sort by negative similarity.
    top_k = 10 # Top k items will be displayed.
    similarity = np.dot(valid_embeddings.cpu().numpy(), normalized_embeddings.cpu().numpy().T)

    # Invert similarity matrix to negative
    # Ignore the first one because that would be the same word as the probe word
    similarity_top_k = np.argsort(-similarity, axis=1)[:, 1: top_k+1]

    # Print the output.
    for i, word_id in enumerate(valid_ids):
        # j >= 1 here since we don't want to include the word itself.
        similar_word_str = ', '.join([itos[j] for j in similarity_top_k[i, :] if j >= 1])
        print(f"{itos[word_id]}: {similar_word_str}")

    print('\n')

### Set up the model

In [ ]:
LR = 10.0
NUM_EPOCHS = 5
EMBED_DIM = 300

In [ ]:
model = SkipGramNegativeSampling(len(VOCAB), EMBED_DIM).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
# The learning rate is lowered every epoch by 1/10.
# Is this a good idea?
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [ ]:
model

SkipGramNegativeSampling(
  (A): Embedding(10996, 300)
  (B): Embedding(10996, 300)
)

In [ ]:
validate_embeddings(model, valid_ids, itos)

women: architecture, accolades, persian, dog, blessed, practicing, comedy, fc, parker, apart
men: emergency, pritzker, increased, markers, slave, not, internews, coowner, downstate, urban
nurse: australian, disorders, matter, frozen, proud, lancet, regulations, kevin, dangerous, felix
doctor: meet, we, data, oxford, baird, yet, anadolu, jurisdictions, caregivers, ama
engineer: unf, contributor, moves, spirits, vehicles, exclusively, musical, dillon, loans, reduction
teacher: induced, reid, needed, munson, breakout, points, thread, farm, click, continental
dietitian: rhode, notably, nationalism, un, gerima, moderate, showcase, gentle, who, rocket




### Train the model

In [ ]:
def train(dataloader, model, optimizer, epoch):
    model.train()
    total_acc, total_count, total_loss, total_batches = 0, 0, 0.0, 0.0
    log_interval = 500

    for idx, x_batch in tqdm(enumerate(dataloader)):

        x_batch = x_batch[0]

        batch_size = x_batch.shape[0]

        optimizer.zero_grad()

        logits = model(x_batch)

        # Get the positive samples loss. Notice we use weights here
        positive_loss = torch.nn.BCEWithLogitsLoss()(input=logits, target=torch.ones(batch_size).to(DEVICE).float())

        # For each batch, get some negative samples
        # We need a total of B * 2 * WINDOW * K samples across a batch
        # We then reshape this batch
        # These are effectively the output words
        negative_samples = torch.multinomial(p, batch_size * K, replacement=True)

        # Repeat the center word enough times so we can merge as needed
        # Repeat does not work on MPS apparently
        wc = x_batch[:, 0].to("cpu").repeat(K).sort().values
        wo = negative_samples

        # Get the negative samples
        x_batch_negative = torch.stack([wc, wo], axis=1).to(DEVICE)

        """
        Note the way we formulated the targets: they are all 0 since these are negative samples.
        We do the BCEWithLogitsLoss by hand basically here.
        Notice we sum across the negative samples, per positive word.

        This is literally the equation in the lecture notes.
        """
        negative_loss = model(x_batch_negative).neg().sigmoid().log().reshape(
            batch_size, K
        ).sum(1).mean().neg().to(DEVICE)

        loss = (positive_loss + negative_loss).mean()

        # Get the gradients via back propagation
        loss.backward()

        # Clip the gradients? Generally a good idea
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)

        # Do an optimization step. Update the parameters A and B
        optimizer.step()
        total_loss += loss.item()
        total_batches += 1

        if idx % log_interval == 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| loss {:8.3f} ".format(
                    epoch,
                    idx,
                    len(dataloader),
                    total_loss / total_batches
                )
            )
            validate_embeddings(model, valid_ids, itos)
            total_loss, total_batches = 0.0, 0.0

### Some results from the run look like below:

In [ ]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()

    train(train_dl, model, optimizer, epoch)
    # We have a learning rate scheduler here
    # Basically, given the state of the optimizer, this lowers the learning rate in a smart way
    scheduler.step()

1it [00:03,  3.22s/it]

| epoch   1 |     0/83264 batches | loss    4.551 
women: architecture, accolades, persian, dog, blessed, practicing, comedy, fc, parker, apart
men: emergency, pritzker, increased, markers, slave, not, internews, coowner, downstate, urban
nurse: australian, disorders, matter, frozen, proud, lancet, regulations, kevin, dangerous, felix
doctor: meet, we, data, oxford, baird, yet, anadolu, jurisdictions, caregivers, ama
engineer: unf, contributor, moves, spirits, vehicles, exclusively, musical, dillon, loans, reduction
teacher: induced, reid, needed, munson, breakout, points, thread, farm, click, continental
dietitian: rhode, notably, nationalism, un, gerima, moderate, showcase, gentle, who, rocket




527it [00:05, 231.18it/s]

| epoch   1 |   500/83264 batches | loss    4.127 
women: architecture, accolades, persian, dog, practicing, blessed, quartet, comedy, fc, gamma
men: emergency, pritzker, increased, markers, not, slave, urban, coowner, internews, downstate
nurse: australian, disorders, lancet, frozen, matter, proud, regulations, kevin, dangerous, knows
doctor: meet, we, data, baird, oxford, anadolu, yet, jurisdictions, ama, gratitude
engineer: unf, contributor, moves, spirits, exclusively, vehicles, dillon, reduction, impairment, musical
teacher: induced, reid, munson, needed, breakout, thread, points, click, muslims, advisory
dietitian: rhode, nationalism, notably, gerima, un, moderate, showcase, rocket, gentle, sidney




1018it [00:07, 237.75it/s]

...

82523it [05:24, 226.31it/s]

| epoch   5 | 82500/83264 batches | loss    1.621 
women: the, america, cultural, black, and, recently, in, of, project, gender
men: featured, life, emergency, designed, dedicated, colleges, private, person, range, dementia
nurse: practitioner, nursing, diverse, cooperates, affiliates, specialists, hospitals, honors, collaborates, doctors
doctor: college, completed, services, tooth, bridges, from, provided, dental, medicine, of
engineer: software, engineering, computer, worked, architect, web, applications, senior, data, systems
teacher: teachers, elementary, education, curriculum, students, learning, teaching, taught, educational, instruction
dietitian: wellness, nutrition, dietetics, affiliates, cooperates, having, specialists, experiences, practitioner, groups




83036it [05:26, 234.28it/s]

| epoch   5 | 83000/83264 batches | loss    1.623 
women: the, america, cultural, black, and, recently, in, of, project, gender
men: featured, life, emergency, designed, dedicated, colleges, private, person, range, dementia
nurse: practitioner, nursing, diverse, cooperates, affiliates, specialists, hospitals, honors, collaborates, doctors
doctor: college, completed, services, tooth, bridges, from, provided, dental, medicine, of
engineer: software, engineering, computer, worked, architect, web, applications, senior, data, systems
teacher: teachers, elementary, education, curriculum, students, learning, teaching, taught, educational, instruction
dietitian: wellness, nutrition, dietetics, affiliates, cooperates, having, specialists, experiences, practitioner, groups




83264it [05:30, 252.30it/s]


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'nurse'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between nurse and average female-related words: 0.31506753
Similarity between nurse and average male-related words: 0.337553


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'doctor'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between doctor and average female-related words: 0.33209607
Similarity between doctor and average male-related words: 0.36901176


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'teacher'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between teacher and average female-related words: 0.42620322
Similarity between teacher and average male-related words: 0.44099373


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'professor'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between professor and average female-related words: 0.3791307
Similarity between professor and average male-related words: 0.371726


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'engineer'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between engineer and average female-related words: 0.30132416
Similarity between engineer and average male-related words: 0.33474162


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'journalist'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between journalist and average female-related words: 0.48601
Similarity between journalist and average male-related words: 0.45716798


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'dietitian'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between dietitian and average female-related words: 0.25119954
Similarity between dietitian and average male-related words: 0.31242508


In [ ]:
# Define the words and their embeddings
female_words = ['woman', 'girl', 'female', 'she', 'her', 'daughter', 'mother', 'wife', 'mrs', 'miss']
male_words = ['man', 'boy', 'male', 'he', 'him', 'his', 'son', 'father', 'husband', 'sir', 'mr']
word = 'dentist'

# Get the embeddings of the female-related words and nurse
female_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in female_words]
embedding = model.A.weight[stoi[word]].detach().cpu().numpy()

# Get the embeddings of the male-related words and nurse
male_embeddings = [model.A.weight[stoi[word]].detach().cpu().numpy() for word in male_words]

# Calculate the average embedding for female-related words
avg_female_embedding = np.mean(female_embeddings, axis=0)

# Calculate the average embedding for male-related words
avg_male_embedding = np.mean(male_embeddings, axis=0)

# Calculate cosine similarity between nurse and average female embedding
similarity_female = np.dot(embedding, avg_female_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_female_embedding)
)

# Calculate cosine similarity between nurse and average male embedding
similarity_male = np.dot(embedding, avg_male_embedding) / (
    np.linalg.norm(embedding) * np.linalg.norm(avg_male_embedding)
)

print(f"Similarity between {word} and average female-related words:", similarity_female)
print(f"Similarity between {word} and average male-related words:", similarity_male)

Similarity between dentist and average female-related words: 0.26894
Similarity between dentist and average male-related words: 0.37059054
